<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/metabolism_Discriptor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors

In [3]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res


In [4]:
# load data
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [5]:
full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37,<rdkit.Chem.rdchem.Mol object at 0x7ff93db2c040>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47,<rdkit.Chem.rdchem.Mol object at 0x7ff93db2c0b0>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45,<rdkit.Chem.rdchem.Mol object at 0x7ff93db2c120>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60,<rdkit.Chem.rdchem.Mol object at 0x7ff93db2c190>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43,<rdkit.Chem.rdchem.Mol object at 0x7ff93db2c200>
...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2,1,7,4.207,55.13,<rdkit.Chem.rdchem.Mol object at 0x7ff93db99bd0>
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5,0,1,-1.736,70.16,<rdkit.Chem.rdchem.Mol object at 0x7ff93db99c40>
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3,1,3,1.792,69.72,<rdkit.Chem.rdchem.Mol object at 0x7ff93db99cb0>
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3,2,2,0.423,69.64,<rdkit.Chem.rdchem.Mol object at 0x7ff93db99d20>


In [6]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

# SMILES로부터 분자 객체 생성
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

# 화합물의 특성 벡터 생성 (여기서는 Morgan fingerprint 사용)
full['Fingerprint'] = full['Molecule'].apply(lambda mol: AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024))

fps = list(full['Fingerprint'])
distance_matrix = []
for i in range(len(full)):
    similarities = []
    for j in range(len(full)):
        sim = DataStructs.TanimotoSimilarity(fps[i], fps[j])
        similarities.append(sim)
    distance_matrix.append(similarities)

# distance matrix를 데이터프레임으로 변환
distance_df = pd.DataFrame(distance_matrix, columns=full.index, index=full.index)

In [21]:
Data = pd.concat([full, full_Descrs], axis=1).drop(columns=['AlogP', 'Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

train = Data.iloc[:3498].dropna(axis=0)
test = Data.iloc[3498:]

In [22]:
train

,id,SMILES,MLM,HLM,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,400.495,5,2,8,3.259,117.37,...,0,0,0,0,0,1,0,0,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,301.407,2,1,2,2.172,73.47,...,0,0,0,0,0,1,0,0,0,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,297.358,5,0,3,1.585,62.45,...,0,0,0,0,1,0,0,0,0,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,494.652,6,0,5,3.475,92.60,...,0,1,0,0,0,0,0,0,0,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,268.310,3,0,1,2.337,42.43,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,396.195,3,1,5,3.409,64.74,...,0,0,0,0,0,0,0,0,0,0
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,359.381,4,1,3,1.844,77.37,...,0,0,0,0,0,0,0,0,0,0
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,261.320,3,1,6,2.124,70.14,...,0,0,0,0,0,0,0,0,0,0
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,284.696,5,1,5,0.989,91.51,...,0,0,0,0,0,0,0,0,0,0


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3496 entries, 0 to 3497
Columns: 215 entries, id to fr_urea
dtypes: float64(108), int64(104), object(3)
memory usage: 5.8+ MB


In [24]:
#결측 확인
for col in train.columns:
    has_missing_values = train[col].isna().any()
    count =  train[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

In [25]:
for col in test.columns:
    has_missing_values = test[col].isna().any()
    count =  test[col].isna().sum()
    if has_missing_values:
        print(f"Column '{col}' has missing values.")
        print(f"'{count}'")

Column 'MLM' has missing values.
'483'
Column 'HLM' has missing values.
'483'


In [27]:
# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

<ipython-input-27-b0c2fd362c62>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[features] = scaler.transform(test[features])


In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train)):
    df_train = train.iloc[train_index]
    df_valid = train.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    # Create and compile the model
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    # Create and compile another model for HLM
    model_hlm = Sequential()
    model_hlm.add(Dense(512, activation='relu', input_shape=(x_train.shape[1],)))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(1024, activation='relu'))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(512, activation='relu'))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(256, activation='relu'))
    model_hlm.add(BatchNormalization())
    model_hlm.add(Dropout(0.2))
    model_hlm.add(Dense(1))

    model_hlm.compile(optimizer=Adam(), loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit(x_train, y_mlm_train, epochs=500, batch_size=32, verbose=1, validation_data=(x_valid, y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit(x_train, y_hlm_train, epochs=500, batch_size=32, verbose=1, validation_data=(x_valid, y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict(x_valid)
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict(x_valid)
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
99/99 [==============================] - 2s 8ms/step - loss: 2414.2319 - root_mean_squared_error: 49.1348 - val_loss: 2415.2471 - val_root_mean_squared_error: 49.1452
Epoch 2/500
99/99 [==============================] - 1s 6ms/step - loss: 2034.9437 - root_mean_squared_error: 45.1104 - val_loss: 2349.1094 - val_root_mean_squared_error: 48.4676
Epoch 3/500
99/99 [==============================] - 1s 6ms/step - loss: 1535.4825 - root_mean_squared_error: 39.1852 - val_loss: 1478.1897 - val_root_mean_squared_error: 38.4472
Epoch 4/500
99/99 [==============================] - 1s 6ms/step - loss: 1085.2142 - root_mean_squared_error: 32.9426 - val_loss: 1085.8398 - val_root_mean_squared_error: 32.9521
Epoch 5/500
99/99 [==============================] - 1s 6ms/step - loss: 911.8690 - root_mean_squared_error: 30.1972 - val_loss: 901.1157 - val_root_mean_squared_error: 30.0186
Epoch 6/500
99/99 [==============================] - 1s 5ms/step - loss: 859.4987 - root_mean_squared_error

In [46]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]

x_test = test[features].values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict(x_test))
    hlm_predictions.append(model_hlm.predict(x_test))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

16/16 [==============================] - 0s 2ms/step


In [47]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [ ]:
# load data
df = test

# predict
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = np.mean([reg_mlm.predict(df[features].values) for reg_mlm in reg_mlms], axis = 0)
df_submission["HLM"] = np.mean([reg_hlm.predict(df[features].values) for reg_hlm in reg_hlms], axis = 0)
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [48]:
df_submission

,id,MLM,HLM
0,TEST_000,21.545601,37.220520
1,TEST_001,58.889080,79.894844
2,TEST_002,32.859509,49.835991
3,TEST_003,52.417084,67.554237
4,TEST_004,54.214989,77.930672
...,...,...,...
478,TEST_478,16.010881,21.149536
479,TEST_479,68.532852,86.734131
480,TEST_480,40.892250,61.705364
481,TEST_481,43.980629,66.921532
